In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import SymLogNorm
from pathlib import Path
import struct
import os
from scipy import fftpack

# Configuration
SIM_RES = 2500
LUX_OUT_DIR = Path('/mnt/home/mlee1/ceph/lux_out')
EXPECTED_RT_GRID = 1024  # RT_GRID size

In [ ]:
# List available lux output directories
print("Lux Output Directories:")
print("=" * 70)

if not LUX_OUT_DIR.exists():
    print(f"Directory not found: {LUX_OUT_DIR}")
else:
    dirs = sorted([d for d in LUX_OUT_DIR.iterdir() if d.is_dir()])
    for d in dirs:
        # Count convergence files
        conv_files = list(d.glob('convergence_*.bin'))
        print(f"{d.name}: {len(conv_files)} convergence maps")

In [ ]:
def read_convergence_file(filepath, grid_size=1024):
    """Read lux convergence map (binary float64 array)."""
    with open(filepath, 'rb') as f:
        data = np.frombuffer(f.read(), dtype=np.float64)
    
    expected_size = grid_size * grid_size
    if len(data) == expected_size:
        return data.reshape((grid_size, grid_size))
    else:
        # Try to infer grid size
        inferred_grid = int(np.sqrt(len(data)))
        if inferred_grid * inferred_grid == len(data):
            return data.reshape((inferred_grid, inferred_grid))
        else:
            raise ValueError(f"Cannot reshape {len(data)} elements to square array")

In [ ]:
# Select a model to analyze
MODEL = 'dmo'  # Change as needed

model_dir = LUX_OUT_DIR / f'L205n{SIM_RES}TNG_{MODEL}'

if not model_dir.exists():
    print(f"Directory not found: {model_dir}")
    print(f"\nAvailable directories:")
    for d in sorted(LUX_OUT_DIR.iterdir()):
        print(f"  {d.name}")
else:
    print(f"Analyzing: {model_dir}")
    
    # List convergence files
    conv_files = sorted(model_dir.glob('convergence_*.bin'))
    print(f"\nFound {len(conv_files)} convergence maps")

In [ ]:
# Load and inspect convergence maps
if conv_files:
    print("\nConvergence Map Statistics:")
    print("-" * 70)
    print(f"{'File':<25} {'Shape':<12} {'Min':<12} {'Max':<12} {'Mean':<12}")
    print("-" * 70)
    
    maps = []
    for f in conv_files[:10]:  # First 10
        try:
            kappa = read_convergence_file(f)
            maps.append((f.name, kappa))
            print(f"{f.name:<25} {str(kappa.shape):<12} {kappa.min():<12.4f} {kappa.max():<12.4f} {kappa.mean():<12.6f}")
        except Exception as e:
            print(f"{f.name:<25} ERROR: {e}")
    
    if len(conv_files) > 10:
        print(f"... and {len(conv_files) - 10} more")

In [ ]:
# Visual inspection of convergence maps
if maps:
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    axes = axes.flatten()
    
    for i, (name, kappa) in enumerate(maps[:6]):
        ax = axes[i]
        
        # Use symmetric log norm for kappa (can be positive or negative)
        vmax = np.percentile(np.abs(kappa), 99)
        im = ax.imshow(kappa, cmap='RdBu_r', 
                       norm=SymLogNorm(linthresh=1e-4, linscale=0.5, vmin=-vmax, vmax=vmax),
                       origin='lower')
        ax.set_title(name)
        plt.colorbar(im, ax=ax, label=r'$\kappa$')
    
    # Turn off unused
    for i in range(len(maps[:6]), 6):
        axes[i].axis('off')
    
    plt.suptitle(f'Convergence Maps - {MODEL}', fontsize=14)
    plt.tight_layout()
    plt.show()

In [ ]:
# Convergence distribution (histogram)
if maps:
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Linear histogram
    ax = axes[0]
    for name, kappa in maps[:5]:
        ax.hist(kappa.flatten(), bins=100, alpha=0.5, label=name, density=True, 
                range=(-0.1, 0.2))
    ax.set_xlabel(r'$\kappa$ (convergence)')
    ax.set_ylabel('Density')
    ax.set_title('Convergence Distribution')
    ax.axvline(0, color='gray', linestyle='--')
    ax.legend(fontsize=8)
    
    # PDF of kappa values
    ax = axes[1]
    all_kappa = np.concatenate([kappa.flatten() for _, kappa in maps])
    ax.hist(all_kappa, bins=200, range=(-0.2, 0.5), density=True, alpha=0.7)
    ax.set_xlabel(r'$\kappa$')
    ax.set_ylabel('PDF')
    ax.set_title(f'Combined PDF ({len(maps)} realizations)')
    ax.axvline(0, color='gray', linestyle='--')
    ax.set_yscale('log')
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Convergence power spectrum
def compute_convergence_power_spectrum(kappa, box_angle_arcmin=60.0):
    """Compute convergence power spectrum.
    
    Args:
        kappa: 2D convergence map
        box_angle_arcmin: Field of view in arcminutes
    
    Returns:
        ell: Multipole moments
        Cl: Power spectrum
    """
    ny, nx = kappa.shape
    
    # FFT
    fft = np.fft.fft2(kappa)
    power = np.abs(fft)**2 / (nx * ny)**2
    
    # Frequencies
    box_rad = np.deg2rad(box_angle_arcmin / 60.0)
    kfreq = np.fft.fftfreq(nx, d=box_rad/nx)
    
    kx, ky = np.meshgrid(kfreq, kfreq)
    k2d = np.sqrt(kx**2 + ky**2)
    ell2d = 2 * np.pi * k2d
    
    # Bin by ell
    ell_bins = np.logspace(1, 5, 50)
    ell_centers = np.sqrt(ell_bins[:-1] * ell_bins[1:])
    
    Cl = np.zeros(len(ell_centers))
    for i in range(len(ell_bins) - 1):
        mask = (ell2d >= ell_bins[i]) & (ell2d < ell_bins[i+1])
        if np.sum(mask) > 0:
            Cl[i] = np.mean(power[mask]) * box_rad**2
    
    return ell_centers, Cl

if maps:
    fig, ax = plt.subplots(figsize=(10, 6))
    
    for name, kappa in maps[:5]:
        ell, Cl = compute_convergence_power_spectrum(kappa)
        ax.loglog(ell, ell * (ell + 1) * Cl / (2 * np.pi), '-', alpha=0.7, label=name)
    
    ax.set_xlabel(r'$\ell$')
    ax.set_ylabel(r'$\ell(\ell+1)C_\ell / 2\pi$')
    ax.set_title(f'Convergence Power Spectrum - {MODEL}')
    ax.legend(fontsize=8)
    ax.grid(True, alpha=0.3)
    ax.set_xlim(100, 1e5)
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Mean power spectrum with error bars
if maps:
    all_Cl = []
    ell_ref = None
    
    for name, kappa in maps:
        ell, Cl = compute_convergence_power_spectrum(kappa)
        all_Cl.append(Cl)
        if ell_ref is None:
            ell_ref = ell
    
    all_Cl = np.array(all_Cl)
    mean_Cl = np.mean(all_Cl, axis=0)
    std_Cl = np.std(all_Cl, axis=0)
    
    fig, ax = plt.subplots(figsize=(10, 6))
    
    y = ell_ref * (ell_ref + 1) * mean_Cl / (2 * np.pi)
    yerr = ell_ref * (ell_ref + 1) * std_Cl / (2 * np.pi)
    
    ax.loglog(ell_ref, y, 'b-', label='Mean')
    ax.fill_between(ell_ref, y - yerr, y + yerr, alpha=0.3, label=f'1σ ({len(maps)} realizations)')
    
    ax.set_xlabel(r'$\ell$')
    ax.set_ylabel(r'$\ell(\ell+1)C_\ell / 2\pi$')
    ax.set_title(f'Mean Convergence Power Spectrum - {MODEL}')
    ax.legend()
    ax.grid(True, alpha=0.3)
    ax.set_xlim(100, 1e5)
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Compare multiple models
models_to_compare = ['dmo', 'hydro', 'replace_Mgt12.5', 'bcm_arico20_Mgt12.5']

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Power spectra
ax = axes[0]
model_spectra = {}

for model in models_to_compare:
    model_dir = LUX_OUT_DIR / f'L205n{SIM_RES}TNG_{model}'
    if model_dir.exists():
        conv_files = sorted(model_dir.glob('convergence_*.bin'))
        if conv_files:
            # Load up to 10 maps
            all_Cl = []
            for f in conv_files[:10]:
                try:
                    kappa = read_convergence_file(f)
                    ell, Cl = compute_convergence_power_spectrum(kappa)
                    all_Cl.append(Cl)
                except:
                    pass
            
            if all_Cl:
                mean_Cl = np.mean(all_Cl, axis=0)
                model_spectra[model] = (ell, mean_Cl)
                y = ell * (ell + 1) * mean_Cl / (2 * np.pi)
                ax.loglog(ell, y, '-', label=f'{model} ({len(all_Cl)})', alpha=0.8)

ax.set_xlabel(r'$\ell$')
ax.set_ylabel(r'$\ell(\ell+1)C_\ell / 2\pi$')
ax.set_title('Power Spectrum Comparison')
ax.legend(fontsize=9)
ax.grid(True, alpha=0.3)
ax.set_xlim(100, 1e5)

# Ratio to DMO
ax = axes[1]
if 'dmo' in model_spectra:
    ell_dmo, Cl_dmo = model_spectra['dmo']
    for model, (ell, Cl) in model_spectra.items():
        if model != 'dmo':
            ratio = Cl / Cl_dmo
            ax.semilogx(ell, ratio, '-', label=model, alpha=0.8)
    
    ax.axhline(1.0, color='gray', linestyle='--')
    ax.set_xlabel(r'$\ell$')
    ax.set_ylabel(r'$C_\ell / C_\ell^{\rm DMO}$')
    ax.set_title('Power Spectrum Ratio to DMO')
    ax.legend(fontsize=9)
    ax.grid(True, alpha=0.3)
    ax.set_xlim(100, 1e5)
    ax.set_ylim(0.8, 1.2)

plt.tight_layout()
plt.show()

In [ ]:
# Summary
print("=" * 70)
print("RAY-TRACING OUTPUT SUMMARY")
print("=" * 70)

for d in sorted(LUX_OUT_DIR.iterdir()):
    if d.is_dir():
        conv_files = list(d.glob('convergence_*.bin'))
        
        if conv_files:
            # Check first file
            try:
                kappa = read_convergence_file(conv_files[0])
                grid = kappa.shape[0]
                mean_kappa = kappa.mean()
                std_kappa = kappa.std()
                status = f"{len(conv_files)} maps, {grid}x{grid}, <κ>={mean_kappa:.2e}"
            except Exception as e:
                status = f"ERROR: {e}"
        else:
            status = "No convergence maps"
        
        print(f"{d.name:<40} {status}")